# Import Moduals

In [1]:
import numpy as np
import pandas as pd
import NeuralNetwork

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.initializers import GlorotUniform
from keras.metrics import CategoricalAccuracy
from keras.losses import MeanSquaredError

from importlib import reload

metrics = CategoricalAccuracy()

# Preprocessing

In [16]:
# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
input_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255
# network parameters
batch_size = 128
hidden_units = 256

11490434/11490434 [==============================] - 0s 0us/step


# Keras' Model

In [62]:
model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size, use_bias=False))
model.add(Dropout(0.45))
model.add(Dense(hidden_units, use_bias=False))
model.add(Dropout(0.45))
model.add(Dense(num_labels, use_bias=False))
model.compile(loss=MeanSquaredError(), 
              optimizer=SGD(learning_rate=0.01),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3853 - accuracy: 0.2419
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1977 - accuracy: 0.3570
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1487 - accuracy: 0.4257
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1238 - accuracy: 0.4741
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1086 - accuracy: 0.5148
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0981 - accuracy: 0.5451
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0903 - accuracy: 0.5742
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0846 - accuracy: 0.5961
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0801 - accuracy: 0.6195
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0768 - accuracy: 0.6339

In [59]:
y_hat = model.predict(x_test)
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

313/313 [==============================] - 0s 926us/step


0.83805

# My Model

In [72]:
reload(NeuralNetwork)
from NeuralNetwork import MyModel, MyLayer, dropout

m = MyModel()
m.addLayer(MyLayer(input_size, hidden_units))
m.addLayer(dropout(0.45))
m.addLayer(MyLayer(hidden_units, hidden_units))
m.addLayer(dropout(0.45))
m.addLayer(MyLayer(hidden_units, num_labels))


m.fit(x_train, y_train, 0.01, epoches = 10, batch_size = batch_size)

Epoch  1: 469/469; Accuracy: 0.2380; Loss: 0.2338: 100%|
Epoch  2: 469/469; Accuracy: 0.3563; Loss: 0.1654: 100%|
Epoch  3: 469/469; Accuracy: 0.4250; Loss: 0.1298: 100%|
Epoch  4: 469/469; Accuracy: 0.4708; Loss: 0.1193: 100%|
Epoch  5: 469/469; Accuracy: 0.5090; Loss: 0.1087: 100%|
Epoch  6: 469/469; Accuracy: 0.5422; Loss: 0.1024: 100%|
Epoch  7: 469/469; Accuracy: 0.5661; Loss: 0.0967: 100%|
Epoch  8: 469/469; Accuracy: 0.5928; Loss: 0.0847: 100%|
Epoch  9: 469/469; Accuracy: 0.6154; Loss: 0.0869: 100%|
Epoch 10: 469/469; Accuracy: 0.6396; Loss: 0.0873: 100%|


In [73]:
y_hat = m.predict(x_test)
metrics.reset_state()
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

0.8426